link to book, the return of sherlock holmes: https://www.gutenberg.org/cache/epub/108/pg108.txt

contributors: Natalie Pedigo, Kylie Quinney

In [ ]:
# install hugging face

%%capture
%pip install transformers[sentencepiece]
import torch
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, pipeline
model_name = "twmkn9/albert-base-v2-squad2"

In [ ]:
import requests

url = "https://www.gutenberg.org/cache/epub/108/pg108.txt"
response = requests.get(url)
book_text = response.text
#remove first 824 chars from book_text
book_text = book_text[1407:10000]
print(book_text[:1000])






THE ADVENTURE OF THE EMPTY HOUSE


      It was in the spring of the year 1894 that all London was
      interested, and the fashionable world dismayed, by the murder of
      the Honourable Ronald Adair under most unusual and inexplicable
      circumstances. The public has already learned those particulars
      of the crime which came out in the police investigation, but a
      good deal was suppressed upon that occasion, since the case for
      the prosecution was so overwhelmingly strong that it was not
      necessary to bring forward all the facts. Only now, at the end of
      nearly ten years, am I allowed to supply those missing links
      which make up the whole of that remarkable chain. The crime was
      of interest in itself, but that interest was as nothing to me
      compared to the inconceivable sequel, which afforded me the
      greatest shock and surprise of any event in my adventurous life.
      Even now, after this long interval, I f


In [ ]:
#tokenizer = AlbertTokenizer.from_pretrained(model_name)
#model = AlbertForQuestionAnswering.from_pretrained(model_name)
#qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
!pip install transformers datasets evaluate
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

In [ ]:
!pip install wandb
import wandb
wandb.login(key="fa5612186849122a2763646da5ad4c5f7342045e")

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    #processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.216438
2,2.596300,1.658516


In [ ]:
from transformers import pipeline, AutoConfig, AutoTokenizer

question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
question = "What year is it?"
context = book_text

result = question_answerer(question=question, context=context)

# Print the answer
print("Answer: " + result['answer'])
#print score
print("Score: " + str(result['score']))

In [ ]:
question = "Who is the main character?"
context = book_text

result = question_answerer(question=question, context=context)

# Print the answer
print("Answer: " + result['answer'])
#print score
print("Score: " + str(result['score']))

In [ ]:
question = "What is the book about?"
context = book_text

result = question_answerer(question=question, context=context)

# Print the answer
print("Answer: " + result['answer'])
#print score
print("Score: " + str(result['score']))